In [1]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

In [2]:
weather = pd.read_csv("weather3_180703.csv")
weather['date'] = pd.to_datetime(weather["date"])
station = weather[weather['station_nbr'] == 20]

In [3]:
station['date'].count()

1035

In [4]:
# ls == 모든 값이 0인 것

ls = []
ls1 = []
for i in station.columns:
    if station[i].count() == 0:
        ls.append(i)
    if station[i].isna().sum() != 0:
        ls1.append(i)
ls, ls1

(['depart', 'sunrise', 'sunset', 'daytime'],
 ['depart', 'sunrise', 'sunset', 'resultdir', 'daytime'])

In [5]:
for i in station.columns:
    count_null = station[i].isna().sum()
    print(i, ":", count_null, "(",round((count_null / len(station) * 100),2),"%",")")

station_nbr : 0 ( 0.0 % )
date : 0 ( 0.0 % )
tmax : 0 ( 0.0 % )
tmin : 0 ( 0.0 % )
tavg : 0 ( 0.0 % )
depart : 1035 ( 100.0 % )
dewpoint : 0 ( 0.0 % )
wetbulb : 0 ( 0.0 % )
heat : 0 ( 0.0 % )
cool : 0 ( 0.0 % )
sunrise : 1035 ( 100.0 % )
sunset : 1035 ( 100.0 % )
codesum : 0 ( 0.0 % )
snowfall : 0 ( 0.0 % )
preciptotal : 0 ( 0.0 % )
stnpressure : 0 ( 0.0 % )
sealevel : 0 ( 0.0 % )
resultspeed : 0 ( 0.0 % )
resultdir : 9 ( 0.87 % )
avgspeed : 0 ( 0.0 % )
year : 0 ( 0.0 % )
month : 0 ( 0.0 % )
day : 0 ( 0.0 % )
relative_humility : 0 ( 0.0 % )
windchill : 0 ( 0.0 % )
daytime : 1035 ( 100.0 % )
week7 : 0 ( 0.0 % )
weekend : 0 ( 0.0 % )
code_change : 0 ( 0.0 % )
rainY : 0 ( 0.0 % )
otherY : 0 ( 0.0 % )
nothing : 0 ( 0.0 % )


In [6]:
train = pd.read_csv("train.csv")
train.date = pd.to_datetime(train.date)
train.tail()

,date,store_nbr,item_nbr,units
4617595,2014-10-31,45,107,0
4617596,2014-10-31,45,108,0
4617597,2014-10-31,45,109,0
4617598,2014-10-31,45,110,0
4617599,2014-10-31,45,111,0


In [7]:
key = pd.read_csv("key.csv")
station = station.merge(key)

In [8]:
station = station.merge(train)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,units
104224,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,NaN,4,0,0,0,0,1,17,107,0
104225,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,NaN,4,0,0,0,0,1,17,108,0
104226,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,NaN,4,0,0,0,0,1,17,109,0
104227,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,NaN,4,0,0,0,0,1,17,110,0
104228,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,NaN,4,0,0,0,0,1,17,111,0


In [9]:
station['log1p_units'] = np.log1p(station.units)

In [10]:
target1 = station['units']
target2 = station['log1p_units']

station.drop(columns=['units','log1p_units'],inplace=True)
station.tail()

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,windchill,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr
104224,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,56.913195,NaN,4,0,0,0,0,1,17,107
104225,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,56.913195,NaN,4,0,0,0,0,1,17,108
104226,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,56.913195,NaN,4,0,0,0,0,1,17,109
104227,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,56.913195,NaN,4,0,0,0,0,1,17,110
104228,20,2014-10-31,68.0,50.0,59.0,NaN,39.0,50.0,6.0,0.0,...,56.913195,NaN,4,0,0,0,0,1,17,111


In [11]:
df1 = pd.concat([station,target1], axis=1)
df2 = pd.concat([station,target2], axis=1)

In [12]:
df2.to_csv('station20.csv', sep=',', index=False)

In [13]:
if df2.columns.tolist() and ls:
    print('공통 변수 있음')
    print(df2.columns.tolist() and ls)
else:
    print('공통 변수 없음')

공통 변수 있음
['depart', 'sunrise', 'sunset', 'daytime']


In [14]:
df2.columns

Index(['station_nbr', 'date', 'tmax', 'tmin', 'tavg', 'depart', 'dewpoint',
       'wetbulb', 'heat', 'cool', 'sunrise', 'sunset', 'codesum', 'snowfall',
       'preciptotal', 'stnpressure', 'sealevel', 'resultspeed', 'resultdir',
       'avgspeed', 'year', 'month', 'day', 'relative_humility', 'windchill',
       'daytime', 'week7', 'weekend', 'code_change', 'rainY', 'otherY',
       'nothing', 'store_nbr', 'item_nbr', 'log1p_units'],
      dtype='object')

In [15]:
ls, ls1

(['depart', 'sunrise', 'sunset', 'daytime'],
 ['depart', 'sunrise', 'sunset', 'resultdir', 'daytime'])

### 1. 변수변환: df2 (log1p_units)

In [17]:
all_var = 'log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(depart) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                            + scale(sunrise) + scale(sunset) + scale(snowfall) + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                            + C(resultdir) + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + C(daytime) + 0'

In [16]:
model2 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                           + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                         + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2)
result2 = model2.fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     4905.
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:54:14   Log-Likelihood:                -3769.1
No. Observations:              104229   AIC:                             7818.
Df Residuals:                  104089   BIC:                             9156.
Df Model:                         139                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(year)[2012]           

### 2. 변수변환 : df2 (log1p_units) + 아웃라이어 제거

In [17]:
# 아웃라이어 제거
# Cook's distance > 2 인 값 제거
influence = result2.get_influence()
cooks_d2, pvals = influence.cooks_distance
fox_cr = 4 / (len(df2) - 2)
idx_outlier = np.where(cooks_d2 > fox_cr)[0]
len(idx_outlier)

1476

In [18]:
idx = list(set(range(len(df2))).difference(idx_outlier))
df2_1 = df2.iloc[idx, :].reset_index(drop=True)
df2_1

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,...,daytime,week7,weekend,code_change,rainY,otherY,nothing,store_nbr,item_nbr,log1p_units
0,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,1,0.000000
1,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,2,0.000000
2,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,3,0.000000
3,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,4,0.000000
4,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,5,0.000000
5,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,6,0.000000
6,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,7,0.000000
7,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,8,0.000000
8,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,9,5.192957
9,20,2012-01-01,59.0,41.0,50.0,NaN,28.0,40.0,15.0,0.0,...,NaN,6,1,0,0,0,1,17,10,0.000000


In [20]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(tmax) + scale(tmin) + scale(tavg) + scale(dewpoint) + scale(wetbulb) + scale(heat) + scale(cool)\
                           + scale(preciptotal) + scale(stnpressure) + scale(sealevel) + scale(resultspeed)  \
                         + scale(avgspeed) + C(year) + C(month) + scale(relative_humility) + scale(windchill) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.260e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:54:40   Log-Likelihood:             1.8396e+05
No. Observations:              102753   AIC:                        -3.676e+05
Df Residuals:                  102613   BIC:                        -3.663e+05
Df Model:                         139                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
C(year)[2012]           

In [21]:
ls1

['depart', 'sunrise', 'sunset', 'resultdir', 'daytime']

### 3. VIF 확인

In [22]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
        'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
        'windchill', 'snowfall' ]

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1388: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


,VIF Factor,features
0,1.372775e+06,stnpressure
1,1.361055e+06,sealevel
2,4.132105e+04,tavg
3,1.601300e+04,windchill
4,7.085491e+03,tmax
5,4.535177e+03,tmin
6,2.910882e+03,wetbulb
7,2.315213e+03,dewpoint
8,6.249503e+02,relative_humility
9,1.004539e+02,cool


### 4. VIF 근거, 빼고 다시 Model 돌림.

In [24]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(heat)\
                         + scale(preciptotal) + scale(resultspeed)  \
                            + scale(avgspeed) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.358e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:55:40   Log-Likelihood:             1.8395e+05
No. Observations:              102753   AIC:                        -3.676e+05
Df Residuals:                  102623   BIC:                        -3.664e+05
Df Model:                         129                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0005      0

### 5. VIF 재확인 

In [25]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# all_cols = ['tmax','tmin','tavg','dewpoint','wetbulb','heat','cool','preciptotal',\
#         'stnpressure','sealevel','resultspeed','avgspeed','relative_humility',\
#         'windchill', 'depart', 'sunrise', 'sunset', 'snowfall' ]
cols = ['heat', 'preciptotal', 'resultspeed', 'avgspeed']

y = df2_1.loc[:,cols]
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(y.values, i) for i in range(y.shape[1])]
vif["features"] = y.columns
vif = vif.sort_values("VIF Factor", ascending=False).reset_index(drop=True)
vif

,VIF Factor,features
0,19.951704,avgspeed
1,19.199488,resultspeed
2,1.383557,heat
3,1.066671,preciptotal


### 6. VIF 근거로 변수 제외하고  Model 돌림.

In [26]:
# OLS - df2_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(heat)\
                            + scale(preciptotal) + C(year) + C(month) + C(weekend) \
                            + C(rainY) + C(store_nbr) + C(item_nbr) + 0', data = df2_1)
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.379e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:56:06   Log-Likelihood:             1.8395e+05
No. Observations:              102753   AIC:                        -3.676e+05
Df Residuals:                  102625   BIC:                        -3.664e+05
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(year)[2012]          0.0004      0

### Anova 분석을 통해 p-value가 0.05보다 큰것은 제외

In [27]:
anova_result2_1 = sm.stats.anova_lm(result2_1).sort_values(by=['PR(>F)'], ascending = False)
anova_result2_1[anova_result2_1['PR(>F)'] <= 0.05]

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


,df,sum_sq,mean_sq,F,PR(>F)
C(rainY),1.0,0.009267,0.009267,5.673804,1.722188e-02
scale(heat),1.0,0.025332,0.025332,15.509455,8.214778e-05
C(weekend),1.0,0.028031,0.028031,17.161668,3.435686e-05
C(month),11.0,1.497126,0.136102,83.326800,1.232725e-188
C(year),3.0,421.386930,140.462310,85996.139339,0.000000e+00
C(item_nbr),110.0,28612.637874,260.114890,159251.804218,0.000000e+00


### 최종 Anova 분석과 TIF 통과한 변수들을 바탕으로 최종 모델생성

In [28]:
# OLS - df2_1_1
model2_1 = sm.OLS.from_formula('log1p_units ~ scale(heat) +  C(rainY) + C(year) + C(month) + \
+ C(item_nbr) + C(weekend) + 0', data = df2_1)

result = model2_1.fit()
result2_1 = model2_1.fit()
print(result2_1.summary())

                            OLS Regression Results                            
Dep. Variable:            log1p_units   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                 1.390e+05
Date:                Fri, 06 Jul 2018   Prob (F-statistic):               0.00
Time:                        13:56:26   Log-Likelihood:             1.8395e+05
No. Observations:              102753   AIC:                        -3.676e+05
Df Residuals:                  102626   BIC:                        -3.664e+05
Df Model:                         126                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(rainY)[0]            0.0004      0

In [29]:
from patsy import dmatrix
# 독립변수와 종속변수로 나누기
df2_1_X = df2_1.drop(columns=['log1p_units'])
df2_1_target = df2_1['log1p_units']

In [32]:
formula = 'scale(heat) + C(rainY) + C(year) + C(item_nbr) + C(weekend) + 0'
dfX = dmatrix(formula, df2_1_X, return_type='dataframe')
dfy = pd.DataFrame(df2_1_target, columns=["log1p_units"])

In [33]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

model = LinearRegression()
cv = KFold(10)

scores = np.zeros(10)
for i, (train_index, test_index) in enumerate(cv.split(dfX)):
    X_train = dfX.values[train_index]
    y_train = dfy.values[train_index]
    X_test = dfX.values[test_index]
    y_test = dfy.values[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[i] = r2_score(y_test, y_pred)

scores

array([0.99377413, 0.99458779, 0.99247362, 0.99382536, 0.99382384,
       0.99432931, 0.99428057, 0.99496592, 0.99374725, 0.99227627])